# ADALINE Assignment
寻找$w^T$权向量使$y=W^T x>7$,$x\in R_1$,

$y=W^T x\leq-3$,$x\in R_2$

$R_1:x\leq1,y\leq1$,$R_2:x\geq3,y\geq2$

## 谢昊君 12000155556 元培学院

# Algorithm及解决思路：
在两个区域随机生成点，然后作为训练集训练，并且每个区域对应不同的损失函数。

error 的设定：

对$R_1$中点,若 $W^T x>7$,$error = 0$

若 $W^T x<7$,$error = (W^T x-7)^2$

对 $R_2$中点 ,若 $W^T x\leq-3$,$error = 0$


若 $W^T x\geq-3$,$error = (W^T x +3)^2$

最终通过更新$w$实现对以下问题的最优化，

$$\min_w\sum(y_{expected} - y_{predicted})^2$$
更新标准:

对error求导，若error = 0，则不更新。

更新标准仍按：

$$w = w + learningrate*(expected - predicted)*feature$$

# 具体实践

使用numpy.random包生成符合要求的随机数据。其中的rand函数生成0到1范围内的随机数，将其乘以100后置于$R_1$，$R_2$中作为训练集。

$R_1$，$R_2$作为一个feature存在于损失函数中，使两区域内的点对应不同的损失函数。

对损失函数做最优化，按更新标准使损失函数最小。

结果：
训练通常在一两次更新后使得所有样本点正确分类。
但因为样本选取的问题，可能并不完美。

In [1]:
import numpy as np
import numpy.random as rd

In [2]:
rd.RandomState()
rd.seed(100)

In [3]:
# Generate the Data from R1 and R2
#R_1
R1 = rd.rand(100,3)*(-100) + 1
R1[:,2]=7
#R_2
R2 = rd.rand(100,3)*(100)
R2[:,0]+=3
R2[:,1]+=2
R2[:,2]= -3

In [4]:
# training set的第三列为常数1，对应常数项。
training_set = np.c_[np.ones(200),np.r_[R1,R2]]

In [5]:
feature = training_set[:,0:3]
output = training_set[:,3]

In [6]:
feature[100]

array([  1.        ,   3.99336937,  80.97656793])

In [14]:
# 定义一个对R1 R2中点分类的函数，其实相当于把R1和R2作为一共feature加入到训练数据中。
def type2(w):
    if ((w[1]<=1)&(w[2]<=1)):
        return "R1"
    elif ((w[1]>=3)&(w[2]>=2)):
        return "R2"

def errors(w,feature,output):
    types =np.array([type2(i) for i in feature])
    tmp = feature.dot(w)-output
    # 对R1，tmp>0,则error为0
    # 对R2，tmp<0,则error为0
    tmp[(types=="R1")&(tmp>0)]=0
    tmp[(types=="R2")&(tmp<0)]=0
    # 此时只输出了绝对误差项，没有计算平方误差，但根据更新标准，是针对平方误差最优化的。
    misclassified = (tmp!=0)
    return (tmp,misclassified)

In [71]:
# Adaline 
# set initial weight and theta
w = np.array([0,0,0])
learning_rate = 0.1
maxiterations = 1000
i = 1
for i in range(1,maxiterations):
    tmp2=errors(w,feature,output)
    error = tmp2[0]
    misclassified = tmp2[1]
    if (error==0).all()==True:
        break
    else:    
    # 选取误分类的第一个点为标准更新:
        w = w + (-1) * learning_rate * (error[misclassified==True][0])*((feature[misclassified==True][0]))
        print(w)

[  0.7        -37.33834593 -18.78585696]


In [83]:
# A class version
class Adaline():
    def __init__(self):
        self.feature = []
        self.output = []
        self.maxiterations = []
        self.learning_rate = []
        self.w = []
        
    def set_training_data(self,feature,output,maxiterations=1000,learning_rate=0.1):
        self.feature = feature
        self.output = output
        self.maxiterations = maxiterations
        self.learning_rate = learning_rate
        
    def errors(w,feature,output):
        types =np.array([type2(i) for i in feature])
        tmp = feature.dot(w)-output
        # 对R1，tmp>0,则error为0
        # 对R2，tmp<0,则error为0
        tmp[(types=="R1")&(tmp>0)]=0
        tmp[(types=="R2")&(tmp<0)]=0
        # 此时只输出了绝对误差项，没有计算平方误差，但根据更新标准，是针对平方误差最优化的。
        misclassified = (tmp!=0)
        return (tmp,misclassified)
    
    def train(self):
        w = rd.rand(3)
        for i in range(1,self.maxiterations):
            tmp2=errors(w,self.feature,self.output)
            error = tmp2[0]
            misclassified = tmp2[1]
            if (error==0).all()==True:
                break
            else:    
    # 选取误分类的第一个点为标准更新:
                w = w + (-1) * learning_rate * (error[misclassified==True][0])*((self.feature[misclassified==True][0]))
        self.w = w
        print(self.w)

In [89]:
# Class Version Test
TMP = Adaline()
TMP.set_training_data(feature,output)
for i in range(1,20):
    TMP.train()

[   5.39984175 -250.7640335  -125.52177341]
[   8.2956452  -428.92589163 -215.34823659]
[   4.80351074 -222.12993448 -111.71699394]
[   5.70873021 -279.72947188 -140.81000513]
[   4.11476952 -208.02745244 -104.24984431]
[   5.104873   -247.12245239 -124.09913331]
[   5.54482179 -253.80903103 -127.125432  ]
[   3.94525676 -164.33735203  -82.14408782]
[   6.66097792 -326.93456363 -164.90386591]
[   5.46746028 -253.41372142 -126.64127928]
[   4.13073045 -201.32521597 -100.35309754]
[   5.11630809 -246.8611776  -123.91808586]
[   4.24706625 -175.74585797  -88.63889762]
[   7.48815259 -348.77602938 -175.12351911]
[   6.77824558 -355.17361774 -178.37405702]
[   6.15678128 -278.68804403 -139.75895115]
[   5.08841917 -224.52598316 -112.69419333]
[   3.43308377 -176.41246436  -88.25339031]
[   3.2926843  -159.55111618  -79.86304732]


In [86]:
feature.dot(w)-output

array([ 2489.50071148,  3100.69491866,  3993.47710504,  3759.30383807,
         833.15599041,  5116.41149082,  3500.09501994,  4983.51608267,
        1292.92636624,  2374.81425467,  3307.85648342,  1432.21284239,
        3712.77818833,  3486.44229532,  2148.47496355,  3416.18779604,
        2606.36023488,  3916.83792677,  1901.4003976 ,   909.36429783,
        2456.07597308,  2224.3226913 ,  2914.9040802 ,  1220.49414611,
        2401.6585049 ,  3544.12175901,  1152.01694179,  1259.83086847,
        2043.02129934,  3342.31230689,  2512.9095893 ,  1963.39741327,
        2091.29496662,  3855.12103214,  2796.92380306,  4321.97259256,
        3896.81496141,  2272.63727881,  1402.61189431,  1817.82684184,
        3151.02267265,   684.27783068,  2107.44403365,  1769.67743596,
        3510.22960544,  2088.7584949 ,  5509.5938112 ,  3402.99377814,
        3912.78827683,  2457.56342988,  1417.4532912 ,  3527.85711429,
        2923.43236165,  1528.5112237 ,  3463.02777383,  3518.25843066,
      

In [82]:
w.shape

(3,)